### Predictive modeling of Orders (Order_payment, review, items and Orders)


In [7]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv
from pathlib import Path
# Carrega as variáveis de ambiente
load_dotenv(dotenv_path=Path('.') / 'environment.env')
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
SF_WAREHOUSE = os.getenv("SF_WAREHOUSE")
SF_DATABASE = os.getenv("SF_DATABASE")
SF_SCHEMA = os.getenv("SF_SCHEMA")

# Conecta ao Snowflake
conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)

# Carrega as tabelas
print("Carregando tabelas do Snowflake...")
df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
df_order_payments = pd.read_sql("SELECT * FROM order_payments_refined", conn)
df_order_items = pd.read_sql("SELECT * FROM order_items_refined", conn)
# 💡 NOVO: Carregando a tabela de produtos
df_products = pd.read_sql("SELECT * FROM products_refined", conn)

# Fecha a conexão
conn.close()

# Padroniza os nomes das colunas
for df in [df_orders, df_orders_reviews, df_order_payments, df_order_items, df_products]:
    df.columns = df.columns.str.lower()

# Realiza as junções sequenciais
print("Unindo as tabelas...")
df_full_orders = df_orders.merge(df_orders_reviews, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_payments, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_items, on='order_id', how='left')
# 💡 NOVO: Juntando a tabela de produtos
df_full_orders = df_full_orders.merge(df_products, on='product_id', how='left')

print("Junção completa. O DataFrame final tem o formato:", df_full_orders.shape)
print(df_full_orders.head())

Carregando tabelas do Snowflake...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order_payments = pd.read_sql("SELECT * FROM o

Unindo as tabelas...
Junção completa. O DataFrame final tem o formato: (119143, 54)
                           order_id                       customer_id  \
0  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
1  85ce859fd6dc634de8d2f1e290444043  059f7fc5719c7da6cbafe370971a8d70   
2  6ea2f835b4556291ffdc53fa0b3b95e8  c7340080e394356141681bd4c9b8fe31   
3  68873cf91053cd11e6b49a766db5af1a  4632eb5a8f175f6fe020520ae0c678f3   
4  8f06cc6465925031568537b815f1198d  9916715c2ab6ee1710c9c32f0a534ad2   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      1511033286000000000  1511034359000000000   
1    delivered      1511222621000000000  1511223262000000000   
2    delivered      1511558868000000000  1511569269000000000   
3    delivered      1512079335000000000  1512183078000000000   
4    delivered      1510745501000000000  1510746402000000000   

  order_delivered_carrier_date order_delivered_customer_date  \
0          15113579990000000

### Treinamento 

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# --- 1. Preparação dos Dados ---
print("Iniciando a preparação dos dados...")

# Limpeza da variável alvo e das features
df_full_orders['review_score'] = pd.to_numeric(df_full_orders['review_score'], errors='coerce')
df_full_orders['delivery_delay_hours'] = pd.to_numeric(df_full_orders['delivery_delay_hours'], errors='coerce')

# 💡 CORREÇÃO: Converte todas as features para tipo numérico ANTES de calcular a mediana
numeric_features = [
    'price',
    'freight_value',
    'payment_installments',
    'total_delivery_time_hours',
    'shipping_time_hours',
    'product_weight_g',
    'product_volume_cm3'
]

for col in numeric_features:
    df_full_orders[col] = pd.to_numeric(df_full_orders[col], errors='coerce')

# Preenche os valores nulos (NaN) com a mediana, uma abordagem comum para evitar distorções
for col in numeric_features:
    median_val = df_full_orders[col].median()
    df_full_orders[col].fillna(median_val, inplace=True)

# Remove as linhas onde a variável alvo é nula
df_full_orders.dropna(subset=['review_score', 'delivery_delay_hours'], inplace=True)


# --- 2. Modelagem para Prever 'review_score' (Regressão) ---
print("\n--- Previsão de Review Score ---")

# Define as features (X) e a variável alvo (y)
X = df_full_orders[numeric_features]
y_score = df_full_orders['review_score']

# Divide os dados em treino e teste
X_train_score, X_test_score, y_train_score, y_test_score = train_test_split(
    X, y_score, test_size=0.2, random_state=42
)

# Treina e avalia os modelos
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=10, random_state=42),
    "XGBoost Regressor": XGBRegressor(n_estimators=10, random_state=42)
}

for name, model in models.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train_score, y_train_score)
    y_pred_score = model.predict(X_test_score)

    r2 = r2_score(y_test_score, y_pred_score)
    mae = mean_absolute_error(y_test_score, y_pred_score)
    rmse = np.sqrt(mean_squared_error(y_test_score, y_pred_score))

    print(f"Métricas para {name}:")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")


# --- 3. Criação da Coluna 'is_satisfied' ---
print("\n--- Criando a coluna 'is_satisfied' ---")
df_full_orders['is_satisfied'] = df_full_orders['review_score'].apply(lambda score: 1 if score >= 4 else 0)
print("Coluna 'is_satisfied' criada com sucesso.")
print(df_full_orders[['review_score', 'is_satisfied']].head())


# --- 4. Modelagem para Prever 'delivery_delay_hours' (Regressão) ---
print("\n--- Previsão de Delivery Delay Hours ---")

# Define as features (X) e a nova variável alvo (y)
X_delay = df_full_orders[numeric_features]
y_delay = df_full_orders['delivery_delay_hours']

# Divide os dados em treino e teste
X_train_delay, X_test_delay, y_train_delay, y_test_delay = train_test_split(
    X_delay, y_delay, test_size=0.2, random_state=42
)

# Treina e avalia os modelos (usando os mesmos modelos de regressão)
for name, model in models.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train_delay, y_train_delay)
    y_pred_delay = model.predict(X_test_delay)

    r2 = r2_score(y_test_delay, y_pred_delay)
    mae = mean_absolute_error(y_test_delay, y_pred_delay)
    rmse = np.sqrt(mean_squared_error(y_test_delay, y_pred_delay))

    print(f"Métricas para {name}:")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

Iniciando a preparação dos dados...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\1665368469.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full_orders[col].fillna(median_val, inplace=True)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\1665368469.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F


--- Previsão de Review Score ---

Treinando Linear Regression...
Métricas para Linear Regression:
R²: 0.0962
MAE: 1.0139
RMSE: 1.2925

Treinando Random Forest Regressor...
Métricas para Random Forest Regressor:
R²: 0.2652
MAE: 0.8533
RMSE: 1.1654

Treinando XGBoost Regressor...
Métricas para XGBoost Regressor:
R²: 0.1489
MAE: 0.9797
RMSE: 1.2543

--- Criando a coluna 'is_satisfied' ---
Coluna 'is_satisfied' criada com sucesso.
   review_score  is_satisfied
0           5.0             1
1           5.0             1
2           1.0             0
3           4.0             1
4           5.0             1

--- Previsão de Delivery Delay Hours ---

Treinando Linear Regression...
Métricas para Linear Regression:
R²: 0.4003
MAE: 138.6131
RMSE: 186.6997

Treinando Random Forest Regressor...
Métricas para Random Forest Regressor:
R²: 0.6046
MAE: 104.7241
RMSE: 151.6012

Treinando XGBoost Regressor...
Métricas para XGBoost Regressor:
R²: 0.5039
MAE: 120.8742
RMSE: 169.8189
